In [1]:
import spacy
import re
#from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

In [13]:
#le nom du fichier contenant le dataset
dataset = "datasets/dataset_hate" 

#le nom du fichier contenant la stoplist
stoplist = "stoplist.txt"

#le nom du fichier dans lequel le dataset nettoyé sera sauvegardé
clean_dataset = "datasets/clean_dataset_hate"

In [3]:
with open(stoplist, "r") as f:
    stopwords = f.read().splitlines()

In [4]:
#importer le vocabulaire français
nlp = spacy.load("fr_core_news_sm")

In [5]:
#Récupération des tweets
with open(dataset, "r") as file:
    raw_data = file.readlines()

In [6]:
# Tokeniser et lemmatiser l'ensemble des tweets
#    -enlever la ponctuation,
#    -enlever les espaces
#    -enlever les email
#    -enlever les urls
def first_clean(raw_data):
    docs = list(nlp.pipe(raw_data))
    first_clean_data = [[token.lemma_.lower() for token in doc 
                         if not token.is_punct and not token.is_space 
                         and not token.like_url and not token.like_email 
                         and token.tag_!= 'Num'] 
                        for doc in docs]
    return first_clean_data

In [7]:
#Deuxième passe de nettoyage
def second_clean(msg):
    clean_msg = []
    for word in msg:
        #Supprimer tout ce qui n'est pas des lettres
        clean_word = re.sub(r'\W', '', word)
        #Supprimer les chiffres
        clean_word = re.sub(r'\d', '', clean_word)
        if clean_word not in stopwords : 
            clean_msg.append(clean_word)
    return clean_msg

In [8]:
def clean(data):
    first_clean_data = first_clean(data)
    clean_data = []
    for msg in first_clean_data:
        clean_data.append(second_clean(msg))
    return clean_data

In [9]:
%%time
# Le nettoyage est fait par paquets de 10000 messages pour éviter de saturer la
# mémoire avec des grands jeux de données
step = 10000
size = 20000 #arrondir au step au dessus
clean_data = []
for i in range(0, size, step):
    print("slice n° {}".format(int((i/step)+1)))
    clean_data += clean(raw_data[i:i+step])

slice n° 1
slice n° 2
CPU times: user 1min 1s, sys: 198 ms, total: 1min 1s
Wall time: 1min 1s


In [11]:
#On a bien uniquement du texte
print(raw_data[0], end='')
print(clean_data[0])

pov tu construis un rideau avec un sac à linge sale et des bd
['pov', 'construire', 'rideau', 'sac', 'linge', 'sale', 'bd']


In [12]:
#enregistrer sur le disque
with open(clean_dataset, "w") as f:
    for msg in clean_data:
        f.write(" ".join(msg) + '\n')